In [85]:
import numpy as np

In [86]:
class Node (object):
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        self.value = None
        
        for n in inbound_nodes:
            n.outbound_nodes.append(self)
        
    def forward(self):
        raise NotImplemented

In [87]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
    
    def forward(self):
        pass

In [93]:
class Linear(Node):
    def __init__(self, inputs, weights, bias):
        Node.__init__(self,[inputs, weights, bias])

    def forward(self):
        inputs = self.inbound_nodes[0].value
        weights = self.inbound_nodes[1].value
        bias = self.inbound_nodes[2]
        self.value = np.dot(inputs, weights) + bias.value
            

In [94]:
def topological_sort(feed_dict):
    input_nodes = [n for n in feed_dict.keys()]
    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in': set(), 'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
    #print("G.elements",G.items())
    L = []
    S=set(input_nodes)
    while(len(S) > 0):
        n = S.pop()
        
        if isinstance(n, Input):
            n.value = feed_dict[n]
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    #print('L', L)
    return L

In [95]:
def forward_pass(ouput_node, sorted_nodes):
    for n in sorted_nodes:
        n.forward()
    return ouput_node.value

In [96]:
X, W, b = Input(), Input(), Input()

X_ = np.array([[-1., -2.], [-1, -2]])
W_ = np.array([[2., -3], [2., -3]])
b_ = np.array([-3., -5])

li = Linear(X, W, b)

feed_dict = {X:X_, W:W_, b:b_}

graph = topological_sort(feed_dict)
output = forward_pass(li, graph)
print("output", output)

output [[-9.  4.]
 [-9.  4.]]


According to the miniflow output is 60 6000 64
